# Learning closeness centrality using deep learning model and features extracted via the RCDF matrix rep. of nodes

This notebook is completes the discussion in Example 7.4 in the article.

In [1]:
from mygraph import MyGraph
from helpers import *
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [2]:
# closeness centrality as the target or label data
target_closeness = np.load("fb_media_pages_closeness.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_closeness[0]

array([0.23340831])

In [3]:
# the RCDF matrix as the feature set
"""
ratio = 1.5
starting_length = 1
maximum_length = 35
radius (order) = 3
"""
matrix = np.load("fb_media_pages_RCDF_matrix_r1_3_sta1_max70_rad3.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [4]:
target_closeness.shape, matrix.shape

((27917, 1), (27917, 3, 23))

In [5]:
matrix[11]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 3., 2., 4., 2., 1., 1., 3., 2., 1., 3., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.]])

# The p-aggregation of the RCDF matrix

To apply this aggregation we just need to multiply the following row matrix from left to the RCDF matrix:
### $$
\left[ 1, p, p^2, \cdots, p^{(r -1 )} \right],
$$
where $r$ is the order of the RCDF matrix representation (the number of rows).

In [6]:
p = 0.2
order = matrix.shape[1]
parameter_vector = np.zeros(order)
for i in range(order):
    parameter_vector[i] += p**i
parameter_vector

array([1.  , 0.2 , 0.04])

In [7]:
feature_vectors = np.matmul(parameter_vector, matrix)
feature_vectors.shape

(27917, 23)

# Converting the dataset into pytorch tensors

In [8]:
features = torch.from_numpy(feature_vectors)
targets = torch.from_numpy(target_closeness)
features.shape, targets.shape

(torch.Size([27917, 23]), torch.Size([27917, 1]))

In [9]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[0.2334],
        [0.2231],
        [0.2180],
        ...,
        [0.2319],
        [0.2520],
        [0.1761]], dtype=torch.float64, requires_grad=True)

# Dividing dataset into train and test data

In [10]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 17917
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([10000, 23]),
 torch.Size([17917, 23]),
 torch.Size([10000, 1]),
 torch.Size([17917, 1]))

## A function for dividing train data into batches

In [11]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [12]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.3)
        
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc2(X))
        return self.fc3(X)

In [13]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=23, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
)

In [14]:
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print(f'Number of all parameters: {num_para}')

1472
64
512
8
8
1
Number of all parameters: 2065


In [15]:
# Objective function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the training loop

In [16]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [17]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   0.01078247219,    TEST LOSS:   0.00293694925
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.00056774908,    TEST LOSS:   0.00063546508
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.00046853322,    TEST LOSS:   0.00044036671
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.00030030753,    TEST LOSS:   0.00035433084
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.00027393666,    TEST LOSS:   0.00030653024
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.00026477949,    TEST LOSS:   0.00026700454
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.00019797157,    TEST LOSS:   0.00023080374
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.00016382299,    TEST LOSS:   0.00020490415
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [18]:
# Saving the losses as a numpy file
np.save("fb_media_pages_train_losses_rcdf_p2.npy", losses[0], allow_pickle=False, fix_imports=True)
np.save("fb_media_pages_test_losses_rcdf_p2.npy", losses[1], allow_pickle=False, fix_imports=True)

In [19]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,23)
    pred = model(feat).item()
    if targ == 0:
        inaccuracy = 0
    else:
        inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%500 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------")

target 0.24643553986299893,    prediction: 0.24229486286640167
index 0:       inaccuracy: 1.68%
-----------------------------------
target 0.23255852733334792,    prediction: 0.23051562905311584
index 500:       inaccuracy: 0.878%
-----------------------------------
target 0.22780430110435182,    prediction: 0.22664013504981995
index 1000:       inaccuracy: 0.511%
-----------------------------------
target 0.1979575919142029,    prediction: 0.19539232552051544
index 1500:       inaccuracy: 1.296%
-----------------------------------
target 0.23557708567516586,    prediction: 0.232253760099411
index 2000:       inaccuracy: 1.411%
-----------------------------------
target 0.24766873429176317,    prediction: 0.244331493973732
index 2500:       inaccuracy: 1.347%
-----------------------------------
target 0.23470159950747266,    prediction: 0.231723353266716
index 3000:       inaccuracy: 1.269%
-----------------------------------
target 0.2044322711786805,    prediction: 0.2019667625427246

In [20]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  1.755


In [21]:
torch.save(model, "trained_model_closeness_fb_media_rcdf_p2.pt")